在进行数据分析和建模的过程中，大量的时间花在数据准备上：加载、清理、转换和重新排列。这样的工作占用了分析师80％以上的时间。有时，数据存储在文件或数据库中的方式对于特定的任务来说格式并不正确。许多研究人员选择使用通用编程语言（如Python、Perl、R或Java）或Unix文本处理工具（如sed或awk）进行从一种形式到另一种形式的特殊数据处理。幸运的是，pandas以及内置的Python语言功能为你提供了一个高级、灵活和快速的工具集，能够将数据处理为正确的形式。

如果发现某种数据处理即不在讨论范围之内，也不在pandas库中，请随时在Python邮件列表或pandas的GitHub页面上分享案例。事实上，pandas中的大部分设计和实现都是由真实世界的应用需求所驱动的。

下面将讨论用于缺失值、重复值、字符串操作和其他分析数据转换的工具。

In [1]:
import numpy as np
import pandas as pd

## 7.1　处理缺失值
缺失数据会在很多数据分析应用中出现。pandas的目标之一就是尽可能无痛地处理缺失值。例如，pandas对象的所有描述性统计信息默认情况下是排除缺失值的。

pandas对象中表现缺失值的方式并不完美，但是它对大部分用户来说是有用的。对于数值型数据，pandas使用浮点值`NaN`（Not a Number来表示缺失值）。称`NaN`为容易检测到的标识值：

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

在pandas中，采用了R语言中的编程惯例，将缺失值成为`NA`，意思是not available（不可用）。在统计学应用中，`NA`数据可以是不存在的数据或者是存在但不可观察的数据（例如在数据收集过程中出现了问题）。当清洗数据用于分析时，对缺失数据本身进行分析以确定数据收集问题或数据丢失导致的数据偏差通常很重要。

Python内建的`None`值在对象数组中也被当作NA处理：

In [4]:
string_data[0] = None

string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

pandas项目持续改善处理缺失值的内部细节，但是用户API函数，比如`pandas.isnull`，抽象掉了很多令人厌烦的细节。下表是处理缺失值的相关函数列表。

| 函数名   | 描述                                                                |
| -       | -                                                                   |
| dropna  | 根据每个标签的值是否是缺失数据来筛选轴标签,并根据允许丢失的数据量来确定阈值 |
| fillna  | 用某些值填充缺失的数据或使用插值方法(如'ffill或'bfill')                 |
| isnull  | 返回表明哪些值是缺失值的布尔值                                         |
| notnull | isnull的反函数                                                       |

### 7.1.1　过滤缺失值
有多种过滤缺失值的方法。虽然可以使用`pandas.isnull`和布尔值索引手动地过滤缺失值，但`dropna`在过滤缺失值时是非常有用的。在Series上使用`dropna`，它会返回Series中所有的非空数据及其索引值：

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

上面的例子与下面的代码是等价的：

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

当处理DataFrame对象时，事情会稍微更复杂一点。可能想要删除全部为`NA`或包含有`NA`的行或列。`dropna`默认情况下会删除包含缺失值的行：

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


传入`how='all'`时，将删除所有值均为`NA`的行：

In [9]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


如果要用同样的方式去删除列，传入参数`axis=1`：

In [10]:
data[4] = NA

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


过滤DataFrame的行的相关方法往往涉及时间序列数据。假设你只想保留包含一定数量的观察值的行。你可以用thresh参数来表示：

In [12]:
df = pd.DataFrame(np.random.randn(7, 3))

df.iloc[:4, 1] = NA

df.iloc[:2, 2] = NA

df

,0,1,2
0,0.107259,NaN,NaN
1,-0.265356,NaN,NaN
2,-0.358923,NaN,-0.368802
3,-0.629510,NaN,0.145532
4,-0.316972,0.246758,0.903203
5,-0.531810,0.381332,0.528626
6,0.148565,-0.018626,-3.163428


In [13]:
df.dropna()

,0,1,2
4,-0.316972,0.246758,0.903203
5,-0.531810,0.381332,0.528626
6,0.148565,-0.018626,-3.163428


In [14]:
df.dropna(thresh=2)

,0,1,2
2,-0.358923,NaN,-0.368802
3,-0.629510,NaN,0.145532
4,-0.316972,0.246758,0.903203
5,-0.531810,0.381332,0.528626
6,0.148565,-0.018626,-3.163428


### 7.1.2　补全缺失值
有时可能需要以多种方式补全“漏洞”，而不是过滤缺失值（也可能丢弃其他数据）。大多数情况下，主要使用`fillna`方法来补全缺失值。调用`fillna`时，可以使用一个常数来替代缺失值：

In [15]:
df.fillna(0)

,0,1,2
0,0.107259,0.000000,0.000000
1,-0.265356,0.000000,0.000000
2,-0.358923,0.000000,-0.368802
3,-0.629510,0.000000,0.145532
4,-0.316972,0.246758,0.903203
5,-0.531810,0.381332,0.528626
6,0.148565,-0.018626,-3.163428


在调用fillna时使用字典，可以为不同列设定不同的填充值：

In [16]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.107259,0.500000,0.000000
1,-0.265356,0.500000,0.000000
2,-0.358923,0.500000,-0.368802
3,-0.629510,0.500000,0.145532
4,-0.316972,0.246758,0.903203
5,-0.531810,0.381332,0.528626
6,0.148565,-0.018626,-3.163428


`fillna`返回的是一个新的对象，但也可以修改已经存在的对象：

In [17]:
_ = df.fillna(0, inplace=True)

df

,0,1,2
0,0.107259,0.000000,0.000000
1,-0.265356,0.000000,0.000000
2,-0.358923,0.000000,-0.368802
3,-0.629510,0.000000,0.145532
4,-0.316972,0.246758,0.903203
5,-0.531810,0.381332,0.528626
6,0.148565,-0.018626,-3.163428


用于重建索引的相同的插值方法也可以用于`fillna`：

In [18]:
df = pd.DataFrame(np.random.randn(6, 3))

df.iloc[2:, 1] = NA

df.iloc[4:, 2] = NA

df

,0,1,2
0,0.792982,-1.742864,-0.443365
1,0.939313,1.372349,-0.765322
2,0.004324,NaN,-1.597421
3,0.237349,NaN,0.624384
4,0.537970,NaN,NaN
5,1.554946,NaN,NaN


In [19]:
df.fillna(method='ffill')

df.fillna(method='ffill', limit=1)

,0,1,2
0,0.792982,-1.742864,-0.443365
1,0.939313,1.372349,-0.765322
2,0.004324,1.372349,-1.597421
3,0.237349,NaN,0.624384
4,0.537970,NaN,0.624384
5,1.554946,NaN,NaN


In [20]:
df.fillna(method='ffill', axis=1, limit=1)

,0,1,2
0,0.792982,-1.742864,-0.443365
1,0.939313,1.372349,-0.765322
2,0.004324,0.004324,-1.597421
3,0.237349,0.237349,0.624384
4,0.537970,0.537970,NaN
5,1.554946,1.554946,NaN


使用`fillna`可以完成很多带有一点创造性的工作。例如，可以将Series的平均值或中位数用于填充缺失值：

In [21]:
data = pd.Series([1., NA, 3.5, NA, 7])

data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

下表是`fillna`的参考。

| 函数名   | 描述                                 |
| -       | -                                    |
| value   | 标量值或字典型对象用于填充缺失值        |
| method  | 插值方法,如果没有其他参数,默认是'ffill' |
| axis    | 需要填充的轴,默认axis=0               |
| inplace | 修改被调用的对象,而不是生成一个备份     |
| limit   | 用于前向或后向填充时最大的填充范围      |

## 7.2　数据转换
目前为止，主要讲解了数据的重新排列。过滤、清洗以及其他转换是另外一系列重要的操作。

### 7.2.1　删除重复值
由于各种原因，DataFrame中会出现重复行。请看如下例子：

In [22]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的`duplicated`方法返回的是一个布尔值Series，这个Series反映的是每一行是否存在重复（与之前出现过的行相同）情况：

In [23]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicates返回的是DataFrame，内容是`duplicated`返回数组中为False的部分：

In [24]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这些方法默认都是对列进行操作。可以指定数据的任何子集来检测是否有重复。假设有一个额外的列，并想基于`'k1'`列去除重复值：

In [25]:
data['v1'] = range(7)

data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated`和`drop_duplicates`默认都是保留第一个观测到的值。传入参数`keep='last'`将会返回最后一个：

In [26]:
data.drop_duplicates(['k1'], keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


### 7.2.2　使用函数或映射进行数据转换
对于许多数据集，可能希望基于DataFrame中的数组、列或列中的数值进行一些转换。考虑下面这些收集到的关于肉类的假设数据：

In [27]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设想要添加一列用于表明每种食物的动物肉类型。先写下一个食物和肉类的映射：

In [28]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map方法接收一个函数或一个包含映射关系的字典型对象，但是这里我们有一个小的问题在于一些肉类大写了，而另一部分肉类没有。因此，我们需要使用Series的str.lower方法将每个值都转换为小写：

In [29]:
lowercased = data['food'].str.lower()

lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [30]:
# 还是没理解好map函数
data['animal'] = lowercased.map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


也可以传入一个能够完成所有工作的函数：

In [31]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用`map`是一种可以便捷执行按元素转换及其他清洗相关操作的方法。

### 7.2.3　替代值
使用`fillna`填充缺失值是通用值替换的特殊案例。前面已经看到，`map`可以用来修改一个对象中的子集的值，但是`replace`提供了更为简单灵活的实现。考虑下面的Series：

In [32]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

`-999`可能是缺失值的标识。如果要使用`NA`来替代这些值，可以使用`replace`方法生成新的Series（除非传入了`inplace=True`）：

In [33]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果想要一次替代多个值，可以传入一个列表和替代值：

In [34]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

要将不同的值替换为不同的值，可以传入替代值的列表：

In [35]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

参数也可以通过字典传递：

In [36]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

> data.replace方法与data.str.replace方法是不同的，data.str.replace是对字符串进行按元素替代的。

### 7.2.4　重命名轴索引
和Series中的值一样，可以通过函数或某种形式的映射对轴标签进行类似的转换，生成新的且带有不同标签的对象。也可以在不生成新的数据结构的情况下修改轴。下面是简单的示例：

In [37]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])   

与Series类似，轴索引也有一个`map`方法：

In [38]:
transform = lambda x: x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

可以赋值给index，修改DataFrame：

In [39]:
data.index = data.index.map(transform)

data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果想要创建数据集转换后的版本，并且不修改原有的数据集，一个有用的方法是`rename`：

In [40]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


值得注意的是，`rename`可以结合字典型对象使用，为轴标签的子集提供新的值：

In [41]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename`可以把手动复制DataFrame并为其分配索引和列属性的烦琐工作中解放出来。如果想要修改原有的数据集，传入`inplace=True`：

In [42]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5　离散化和分箱
连续值经常需要离散化，或者分离成”箱子“进行分析。假设有某项研究中一组人群的数据，想将他们进行分组，放入离散的年龄框中：

In [43]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

将这些年龄分为18～25、26～35、36～60以及61及以上等若干组。为了实现这个，可以使用pandas中的`cut`：

In [44]:
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的对象是一个特殊的`Categorical`对象。看到的输出描述了由`pandas.cut`计算出的箱。可以将它当作一个表示箱名的字符串数组；它在内部包含一个`categories`（类别）数组，指定了不同的类别名称以及`codes`属性中的ages（年龄）数据标签：

In [45]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [46]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [47]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

> pd.value_counts（cats）是对pandas.cut的结果中的箱数量的计数。

与区间的数学符号一致，小括号表示边是开放的，中括号表示它是封闭的（包括边）。可以通过传递`right=False`来改变哪一边是封闭的：

In [48]:
print(pd.cut(ages, [18, 26, 36, 61, 100], right=False))
pd.cut(ages, [18, 26, 36, 61, 100], right=False).value_counts()

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]


[18, 26)     5
[26, 36)     3
[36, 61)     3
[61, 100)    1
dtype: int64

也可以通过向`labels`选项传递一个列表或数组来传入自定义的箱名：

In [49]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

如果传给`cut`整数个的箱来代替显式的箱边，pandas将根据数据中的最小值和最大值计算出**等长**的箱。考虑一些均匀分布的数据被切成四份的情况：

In [50]:
data = np.random.rand(20)

pd.cut(data, 4, precision=2)

[(0.75, 0.97], (0.081, 0.3], (0.75, 0.97], (0.52, 0.75], (0.52, 0.75], ..., (0.3, 0.52], (0.75, 0.97], (0.75, 0.97], (0.081, 0.3], (0.3, 0.52]]
Length: 20
Categories (4, interval[float64]): [(0.081, 0.3] < (0.3, 0.52] < (0.52, 0.75] < (0.75, 0.97]]

In [51]:
pd.cut(data, 4, precision=2).value_counts()

(0.081, 0.3]    3
(0.3, 0.52]     8
(0.52, 0.75]    3
(0.75, 0.97]    6
dtype: int64

> `precision=2`的选项将十进制精度限制在两位。

`qcut`是一个与分箱密切相关的函数，它基于样本分位数进行分箱。取决于数据的分布，使用`cut`通常不会使每个箱具有相同数据量的数据点。由于`qcut`使用样本的分位数，可以通过`qcut`获得等长的箱：

In [52]:
data = np.random.randn(1000)  # 正态分布

cats = pd.qcut(data, 4)  # 切成四份

cats

[(-0.0413, 0.601], (-0.719, -0.0413], (-0.0413, 0.601], (-3.499, -0.719], (-0.719, -0.0413], ..., (-0.0413, 0.601], (-3.499, -0.719], (-3.499, -0.719], (-0.719, -0.0413], (-0.719, -0.0413]]
Length: 1000
Categories (4, interval[float64]): [(-3.499, -0.719] < (-0.719, -0.0413] < (-0.0413, 0.601] < (0.601, 3.603]]

In [53]:
pd.value_counts(cats, ascending=True)

(-3.499, -0.719]     250
(-0.719, -0.0413]    250
(-0.0413, 0.601]     250
(0.601, 3.603]       250
dtype: int64

与`cut`类似，可以传入自定义的分位数**（0和1之间的数据，包括边）**：

In [54]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0413, 1.214], (-1.359, -0.0413], (-0.0413, 1.214], (-1.359, -0.0413], (-1.359, -0.0413], ..., (-0.0413, 1.214], (-1.359, -0.0413], (-1.359, -0.0413], (-1.359, -0.0413], (-1.359, -0.0413]]
Length: 1000
Categories (4, interval[float64]): [(-3.499, -1.359] < (-1.359, -0.0413] < (-0.0413, 1.214] < (1.214, 3.603]]

In [55]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.499, -1.359]     100
(-1.359, -0.0413]    400
(-0.0413, 1.214]     400
(1.214, 3.603]       100
dtype: int64

后续章节中，在讨论聚合和分组操作时，将会继续讨论`cut`和`qcut`，因为这些离散化函数对于分位数和分组分析特别有用。

### 7.2.6　检测和过滤异常值
过滤或转换异常值在很大程度上是应用数组操作的事情。考虑一个具有正态分布数据的DataFrame：

In [56]:
data = pd.DataFrame(np.random.randn(1000, 4))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.032662,0.003678,0.035721,0.006263
std,0.992490,1.030748,1.003890,1.030235
min,-3.091072,-3.168489,-3.013133,-3.120585
25%,-0.643353,-0.733209,-0.643864,-0.669102
50%,0.027093,-0.020149,0.048881,0.025400
75%,0.772507,0.671256,0.716115,0.710845
max,3.384192,3.045431,3.060083,3.547364


想要找出一列中绝对值大于三的值：

In [57]:
col = data[2]

col[np.abs(col) > 3]
# 或者
# col[col.abs()>3]

670   -3.013133
765    3.060083
Name: 2, dtype: float64

要选出所有值大于3或小于-3的行，你可以对布尔值DataFrame使用`any`方法：

In [58]:
data[(np.abs(data) > 3).any(axis=1)]

,0,1,2,3
35,0.666864,2.601230,0.898893,-3.052332
108,-3.091072,-0.040014,-0.880337,-0.214675
194,-0.021220,-3.027087,-0.741916,0.070613
198,0.349168,1.276660,-0.456480,-3.120585
374,-0.241642,-1.133555,-0.082670,3.547364
537,0.099944,-3.168489,-0.254771,1.475983
627,1.962777,-3.117345,-0.834572,-0.189854
649,3.384192,1.304127,-0.815344,-2.162351
670,-0.916439,-0.828663,-3.013133,0.928466
765,-0.701982,-0.280801,3.060083,-0.619982


值可以根据这些标准来设置，下面代码限制了-3到3之间的数值：

In [59]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.032369,0.003945,0.035674,0.005888
std,0.990982,1.029680,1.003671,1.027981
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.643353,-0.733209,-0.643864,-0.669102
50%,0.027093,-0.020149,0.048881,0.025400
75%,0.772507,0.671256,0.716115,0.710845
max,3.000000,3.000000,3.000000,3.000000


语句`np.sign(data)`根据数据中的值的正负分别生成1和-1的数值：

In [60]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,1.0
1,1.0,1.0,-1.0,1.0
2,1.0,1.0,-1.0,1.0
3,1.0,1.0,1.0,-1.0
4,1.0,1.0,-1.0,1.0


### 7.2.7　置换和随机抽样
使用`numpy.random.permutation`对DataFrame中的Series或行进行置换（随机重排序）是非常方便的。在调用`permutation`时根据想要的轴长度可以产生一个表示新顺序的整数数组：

In [61]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [62]:
sampler = np.random.permutation(5)

sampler

array([0, 2, 4, 3, 1])

整数数组可以用在基于`iloc`的索引或等价的`take`函数中：

In [63]:
# 打乱数据顺序
df.take(sampler)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7


要选出一个不含有替代值的随机子集，可以使用Series和DataFrame的sample方法：

In [64]:
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
4,16,17,18,19


要生成一个带有替代值的样本（允许有重复选择），将`replace=True`传入sample方法：

In [65]:
choices = pd.Series([5, 7, -1, 6, 4])

# 不带参数时会报错
draws = choices.sample(n=10, replace=True)

draws

0    5
0    5
4    4
2   -1
4    4
1    7
4    4
0    5
1    7
3    6
dtype: int64

### 7.2.8　计算指标/虚拟变量
将分类变量转换为“虚拟”或“指标”矩阵是另一种用于统计建模或机器学习的转换操作。如果DataFrame中的一列有k个不同的值，则可以衍生一个k列的值为1和0的矩阵或DataFrame。pandas有一个`get_dummies`函数用于实现该功能，尽管自行实现也不难。回顾一下之前的一个示例DataFrame：

In [66]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [67]:
# 按key字段的内容，进行0/1归一化，分类数据转数字
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


在某些情况下，可能想在指标DataFrame的列上加入前缀，然后与其他数据合并。在`get_dummies`方法中有一个前缀参数用于实现该功能：

In [68]:
dummies = pd.get_dummies(df['key'], prefix='key')

df_with_dummy = df[['data1']].join(dummies)

df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果DataFrame中的一行属于多个类别，则情况略为复杂。看看MovieLens的1M数据集：

In [69]:
mnames = ['movie_id', 'title', 'genres']

movies = pd.read_csv('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames, engine='python')

movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


为每个电影流派添加指标变量需要进行一些数据处理。首先，从数据集中提取出所有不同的流派的列表：

In [70]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))

genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

使用全0的DataFrame是构建指标DataFrame的一种方式：

In [71]:
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix.shape

(3883, 18)

In [72]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

现在，遍历每一部电影，将dummies每一行的条目设置为1。为了实现该功能，使用`dummies.columns`来计算每一个流派的列指标：

In [73]:
gen = movies.genres[0]

gen.split('|')

['Animation', "Children's", 'Comedy']

In [74]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

之后，使用.loc根据这些指标来设置值：

In [75]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

之后，和前面一样，可以将结果与movies进行联合：

In [76]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [77]:
movies_windic.iloc[:1,:]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> 对于更大的数据，上面这种使用多成员构建指标变量并不是特别快速。更好的方法是写一个直接将数据写为NumPy数组的底层函数，然后将结果封装进DataFrame。

将`get_dummies`与`cut`等离散化函数结合使用是统计应用的一个有用方法：

In [78]:
np.random.seed(12345)

values = np.random.rand(10)

values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [79]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
print(pd.cut(values, bins).value_counts())
pd.get_dummies(pd.cut(values, bins))

(0.0, 0.2]    1
(0.2, 0.4]    2
(0.4, 0.6]    2
(0.6, 0.8]    3
(0.8, 1.0]    2
dtype: int64


   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

使用`numpy.random.seed`来设置随机种子以确保示例的确定性。将在后面的内容中再次讨论`pandas.get_dummies`。

## 7.3　字符串操作
由于Python在字符串和文本操作上的便利性，使Python成为一个流行的原生数据集操作语言已经有很长时间了。字符串对象的内建方法使得大部分文本操作非常简单。对于更为复杂的模式匹配和文本操作，正则表达式可能是需要的。pandas允许将字符串和正则表达式简洁地应用到整个数据数组上，此外还能处理数据缺失带来的困扰。

### 7.3.1　字符串对象方法
在很多字符串处理和脚本应用中，内建的字符串方法是足够的。例如，一个逗号分隔的字符串可以使用`split`方法拆分成多块：

In [80]:
val = 'a,b,  guido'

val.split(',')

['a', 'b', '  guido']

> `split`常和`strip`一起使用，用于清除空格（包括换行）：

In [81]:
pieces = [x.strip() for x in val.split(',')]

pieces

['a', 'b', 'guido']

这些子字符串可以使用加法与两个冒号分隔符连接在一起：

In [82]:
first, second, third = pieces

first + '::' + second + '::' + third

'a::b::guido'

但是这并不是一个实用的通用方法。在字符串`'：：'`的`join`方法中传入一个列表或元组是一种更快且更加Pythonic（Python风格化）的方法：

In [83]:
'::'.join(pieces)

'a::b::guido'

其他方法涉及定位子字符串。使用Python的`in`关键字是检测子字符串的最佳方法，尽管`index`和`find`也能实现同样的功能：

In [84]:
'guido' in val

True

In [85]:
val.index(',')

1

In [86]:
val.find(':')

-1

请注意`find`和`index`的区别在于`index`在字符串没有找到时会抛出一个异常（而`find`是返回-1）：

In [87]:
val.index(':')

ValueError: substring not found

相关地，`count`返回的是某个特定的子字符串在字符串中出现的次数：

In [88]:
val.count(',')

2

`replace`将用一种模式替代另一种模式。它通常也用于传入空字符串来删除某个模式。

In [89]:
val.replace(',', '::')

'a::b::  guido'

In [90]:
val.replace(',', '')

'ab  guido'

下表是一些Python字符串方法的列表。

|         方法          |                                   描述                                    |
| --------------------- | ------------------------------------------------------------------------- |
| count                 | 返回子字符串在字符串中的非重叠出现次数                                        |
| endswith              | 如果字符串以后缀结尾则返回True                                               |
| startswith            | 如果字符串以前缀开始则返回True                                               |
| join                  | 使用字符串作为间隔符,用于粘合其他字符串的序列                                  |
| index                 | 如果在字符串中找到,则返回子字符串中第一个字符的位置;如果找不到则引发 ValueError   |
| find                  | 返回字符串中第一个出现子字符的第一个字符的位置;类似`index`,但如果没有找到则返回-1 |
| rfind                 | 返回子字符串在字符串中最后一次出现时第一个字符的位置;如果没有找到,则返回-1        |
| replace               | 使用一个字符串替代另一个字符串                                               |
| strip, rstrip, lstrip | 修剪空白,包括换行符;相当于对每个元素进行x.strip()(以及rstrip, istrip)          |
| split                 | 使用分隔符将字符串拆分为子字符串的列表                                        |
| lower                 | 将大写字母转换为小写字母                                                    |
| upper                 | 将小写字母转换为大写字母                                                    |
| casefold              | 将字符转换为小写,并将任何特定于区域的变量字符组合转换为常见的可比较形式           |
| ljust, rjust          | 左对齐或右对齐;用空格(或其他一些字符)填充字符串的相反侧以返回具有最小宽度的字符串 |

后续将看到正则表达式也可以用于这些方法。

### 7.3.2　正则表达式
正则表达式提供了一种在文本中灵活查找或匹配（通常更为复杂的）字符串模式的方法。单个表达式通常被称为regex，是根据正则表达式语言形成的字符串。Python内建的`re`模块是用于将正则表达式应用到字符串上的库。

> 编写正则表达式的艺术是可以自成一章的。在网上有很多优秀的教程。

re模块主要有三个主题：**模式匹配**、**替代**、**拆分**。当然，这三部分主题是相关联的。一个正则表达式描述了在文本中需要定位的一种模式，可以用于多种目标。来看一个简单的示例：假设想将含有多种空白字符（制表符、空格、换行符）的字符串拆分开。描述一个或多个空白字符的正则表达式是`\s+`：

In [91]:
import re
text = "foo    bar\t baz  \tqux"

re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

当调用`re.split('\s+'，text)`，正则表达式首先会被编译，然后正则表达式的`split`方法在传入文本上被调用。可以使用`re.compile`自行编译，形成一个可复用的正则表达式对象：

In [92]:
regex = re.compile('\s+')

regex.split(text)

['foo', 'bar', 'baz', 'qux']

如果想获得的是一个所有匹配正则表达式的模式的列表，可以使用findall方法：

In [93]:
regex.findall(text)

['    ', '\t ', '  \t']

> 为了在正则表达式中避免转义符`\`的影响，可以使用原生字符串语法，比如`r'C：\x'`或者用等价的`'C：\\x'`

> 如果需要将相同的表达式应用到多个字符串上，推荐使用`re.compile`创建一个正则表达式对象，这样做有利于节约CPU周期。

`match`和`search`与`findall`相关性很大。`findall`返回的是字符串中所有的匹配项，而`search`返回的仅仅是第一个匹配项。`match`更为严格，它只在字符串的起始位置进行匹配。作为一个不重要的示例，考虑下一段文本以及一个可以识别大部分电子邮件地址的正则表达式：

In [94]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE使正则表达式不区分大小写
regex = re.compile(pattern, flags=re.IGNORECASE)

在文本上使用findall会生成一个电子邮件地址的列表：

In [95]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search`返回的是文本中第一个匹配到的电子邮件地址。对于前面提到的正则表达式，匹配对象只能告诉我们模式在字符串中起始和结束的位置：

In [96]:
m = regex.search(text)

m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [97]:
text[m.start():m.end()], m.start(), m.end()

('dave@google.com', 5, 20)

`regex.match`只在模式出现于字符串起始位置时进行匹配，如果没有匹配到，返回None：

In [98]:
print(regex.match(text))

None


相关地，`sub`会返回一个新的字符串，原字符串中的模式会被一个新的字符串替代：

In [99]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



假设想查找电子邮件地址，并将每个地址分为三个部分：用户名，域名和域名后缀。要实现这一点，可以用`括号`将模式包起来：

In [100]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags=re.IGNORECASE)

由这个修改后的正则表达式产生的匹配对象的`groups`方法，返回的是模式组件的元组：

In [101]:
m = regex.match('wesm@bright.net')

m.groups()

('wesm', 'bright', 'net')

当模式可以分组时，`findall`返回的是包含元组的列表：

In [102]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub`也可以使用特殊符号，如`\1`和`\2`，访问每个匹配对象中的分组。符号`\1`代表的是第一个匹配分组，`\2`代表的是第二个匹配分组，以此类推：

In [103]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



Python中关于正则表达式还有更多的内容。下表提供了一个简要概述。


|    方法    |                                                         描述                                                          |
| --------- | --------------------------------------------------------------------------------------------------------------------- |
| findall   | 将字符串中所有的非重叠匹配模式以列表形式返回                                                                                |
| finditer  | 与 findall类似,但返回的是迭代器                                                                                          |
| match     | 在字符串起始位置匹配模式,也可以将模式组建匹配到分组中;<br>如果模式匹配上了,返回的一个匹配对象,否则返回None                             |
| search    | 扫描字符串的匹配模式,如果扫描到了返回匹配对象,<br>与 match方法不同的是, search方法的匹配可以是字符串的任意位置,<br>而不仅仅是字符串的起始位置 |
| split     | 根据模式,将字符串拆分为多个部分                                                                                           |
| sub, subn | 用替换表达式替换字符串中所有的匹配(sub)或第n个出现的匹配串(subn);<br>使用符号\\1、\\2……来引用替换字符串中的匹配组元素                  |

### 7.3.3　pandas中的向量化字符串函数
清理杂乱的数据集用于分析通常需要大量的字符串处理和正则化。包含字符串的列有时会含有缺失数据，使事情变得复杂：

In [104]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [105]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

可以使用`data.map`将字符串和有效的正则表达式方法（以`lambda`或其他函数的方式传递）应用到每个值上，但是在NA（null）值上会失败。为了解决这个问题，Series有面向数组的方法用于跳过NA值的字符串操作。这些方法通过Series的`str`属性进行调用，例如，可以通过`str.contains`来检查每个电子邮件地址是否含有`'gmail'`：

In [106]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

正则表达式也可以结合任意的re模块选项使用，例如`IGNORECASE`：

In [107]:
pattern

data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

有多种方法可以进行向量化的元素检索。可以使用`str.get`或在`str`属性内部索引：

In [108]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

要访问嵌入式列表中的元素，可以将索引传递给这些函数中的任意一个：

In [109]:
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [110]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

可以使用字符串切片的类似语法进行向量化切片：

In [111]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

下表中有更多pandas字符串方法。

|     方法      |                                          描述                                           |
| ------------ | -------------------------------------------------------------------------------------- |
| cat          | 根据可选的分隔符按元素黏合字符串                                                            |
| contains     | 返回是否含有某个模式/正则表达式的布尔值数组                                                  |
| count        | 模式出现次数的计数                                                                        |
| extract      | 使用正则表达式从字符串Series中分组抽取一个或多个字符串;<br>返回的结果是每个分组形成一列的Data Frame |
| endswith     | 等价于对每个元素使用x.endwith(模式)                                                      |
| startswith   | 等价于对每个元素使用x.startwith(模式)                                                    |
| findall      | 找出字符串中所有的模式/正则表达式匹配项,以列表返回                                            |
| get          | 对每个元素进行索引(获得第i个元素)                                                           |
| isalnum      | 等价于内建的str.alnum                                                                     |
| isalhpa      | 等价于内建的str.isalpha                                                                 |
| isdecimal    | 等价于内建的str.isdecima1                                                               |
| isdigit      | 等价于内建的str.isdigit                                                                 |
| islower      | 等价于内建的str.is1ower                                                                  |
| isnumeric    | 等价于内建的str.isnumeric                                                               |
| isupper      | 等价于内建的str.isupper                                                                 |
| join         | 根据传递的分隔符,将Series中的字符串联合                                                    |
| len          | 计算每个字符串的长度                                                                      |
| lower, upper | 转换大小写;等价于对每个元素进行`x.1ower()`或`x.upper()`                                       |
| match        | 使用`re.match`将正则表达式应用到每个元素上,将匹配分组以列表形式返回                             |
| pad          | 将空白加到字符串的左边、右边或两边                                                          |
| center       | 等价于`pad(side='both')`                                                                |
| repeat       | 重复值(例如`s.str.repeat(3)`等价于对每个字符串进行x\*3)                                      |
| replace      | 以其他字符串替代模式/正式表达式的匹配项                                                      |
| slice        | 对 Series中的字符串进行切片                                                               |
| split        | 以分隔符或正则表达式对字符串进行拆分                                                        |
| strip        | 对字符串两侧的空白进行消除,包括换行符                                                       |
| rstrip       | 消除字符串右边的空白                                                                      |
| lstrip       | 消除字符串左边的空自                                                                      |